# Исследование надёжности заёмщиков

Задача - выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
# импотрируем все необходимые библиотеки для работы над проектом
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
import numpy as np

In [2]:
# считает данные датасета
statistic_score = pd.read_csv('/datasets/data.csv')
statistic_score.info()
statistic_score.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**
В таблице 12 столбцов с корректными названиями и 21 525 строк.
Значения ячеек представлены типами: вещественные числа (float), целочисленные значения (int), строки (либо иной тип данных,  не подходящих под другой тип) (object).
В двух столбцах days_emplyed и total_income, строк одинаковое количество и их меньше чем остальных. Это пропуски, которые вероятно были получены ввиду того, что произошла ошибка обработки данных учета трудового стажа и дохода.

Можно отметить различные названия значений в ячейках, дубликаты, которые разберем в соответствующем разделе.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
# подсчитываем пропуски
statistic_score.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
# найдем уникальные пропущенные значения каждого столбца
columns_unique = ['children', 'days_employed', 'education', 'education_id', 'family_status', 'family_status_id', 
                  'gender', 'income_type', 'debt', 'total_income', 'purpose']
for column in columns_unique:
    print(column, statistic_score[column].unique())
    print()

children [ 1  0  3  2 -1  4 20  5]

days_employed [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]

education ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

education_id [0 1 2 3 4]

family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

family_status_id [0 1 2 3 4]

gender ['F' 'M' 'XNA']

income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

debt [0 1]

total_income [253875.6394526  112080.01410244 145885.95229686 ...  89672.56115303
 244093.05050043  82047.41889948]

purpose ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покуп

In [5]:
# заменим значения столбца education и family_status на нижний регистр и проверим исправление
statistic_score['education'] = statistic_score['education'].str.lower()
statistic_score['family_status'] = statistic_score['family_status'].str.lower()
print(statistic_score['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [6]:
# заменим отрицательные данные на абсолютные и проверим исправление
statistic_score['children'] = abs(statistic_score['children'])
print(statistic_score['children'].unique())
statistic_score['days_employed'] = abs(statistic_score['days_employed'])
print(statistic_score['days_employed'].unique())

[ 1  0  3  2  4 20  5]
[8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]


Можно заменить значение пола XNA на 'unknown' в силу его малозначимости для проверки наших гипотез.
Вероятно, это ошибка техническая, случайная. Данная ошибка единична в датафрейме.


In [7]:
# Проверим количество повторов уникальных значений столбца gender
statistic_score['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [8]:
# Заменим значение XNA на 'unknown'
statistic_score = statistic_score.replace('XNA', 'unknown')
statistic_score['gender'].value_counts()

F          14236
M           7288
unknown        1
Name: gender, dtype: int64

In [9]:
# заменим пропущенные значения столбца трудового стажа на 0 в силу того,
# что для проверки наших гипотез это незначимо
statistic_score['days_employed'] =  statistic_score['days_employed'].fillna(0)
# проверим, все ли пропуски исправлены
statistic_score.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Начнем подготовку к заполнению пропусков в столбце total_income с помощью медианного значения в группах, соответствующих по уровню обрзованию (education), типу занятости (income_type) и возрасту (dob_years).

In [10]:
# создадим сводную таблицу медианных значений каждой строки по столбцу ежемесячных доходов
# (совет наставника по проекту) и выведем ее на экран
median_total_score = statistic_score.pivot_table(index=['dob_years', 'income_type'], columns='education_id', values='total_income', aggfunc='median')
median_total_score.head(10)

education_id                       0              1              2   3   4
dob_years income_type                                                     
0         госслужащий  147473.884045  116853.411466            NaN NaN NaN
          компаньон    224510.589360  167713.212064            NaN NaN NaN
          пенсионер    137674.117012  102621.701671            NaN NaN NaN
          сотрудник    157362.970952  129945.107346  192158.594143 NaN NaN
19        госслужащий            NaN            NaN   75787.415536 NaN NaN
          компаньон              NaN  103676.482282  166240.210560 NaN NaN
          сотрудник              NaN   93343.132708   85955.190475 NaN NaN
20        госслужащий   71105.377210  248725.327979            NaN NaN NaN
          компаньон    100002.327317  144986.205432   86565.423291 NaN NaN
          сотрудник     98287.203541  117511.968537   86927.446323 NaN NaN

In [ ]:
# напишем и применим функцию к данным по несколькиv группам для поиска медианных значений по столбцу total_income
def fillna_total_income(dob_years, income_type, education_id):
    try:
        return median_total_score[education_id][dob_years][income_type]
    except:
        return 'ошибка'
# запишем данные результаты в новый столбец исходного датафрейма
statistic_score['median_total_income'] = statistic_score.apply(lambda row: fillna_total_income(row['dob_years'], row['income_type'],  row['education_id']), axis=1)
# заменим пропуски в столбце 'total_income' из нового столбца полученных медаин
statistic_score['total_income'] = statistic_score['total_income'].fillna(statistic_score['median_total_income']) 
# проверим результат пропусков
statistic_score.isna().sum()

In [12]:
# узнаем, что это за пропуски
statistic_score[statistic_score['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_total_income
1303,1,0.0,70,начальное,3,гражданский брак,1,F,сотрудник,0,NaN,операции с коммерческой недвижимостью,NaN
1520,0,0.0,53,начальное,3,женат / замужем,0,F,сотрудник,0,NaN,операции со своей недвижимостью,NaN
3612,0,0.0,40,высшее,0,женат / замужем,0,M,пенсионер,0,NaN,ремонт жилью,NaN
3891,1,0.0,68,неоконченное высшее,2,в разводе,3,F,сотрудник,0,NaN,строительство недвижимости,NaN
5530,0,0.0,23,высшее,0,не женат / не замужем,4,M,госслужащий,0,NaN,покупка жилья для семьи,NaN
8142,0,0.0,64,начальное,3,гражданский брак,1,F,госслужащий,0,NaN,сыграть свадьбу,NaN
16091,0,0.0,49,неоконченное высшее,2,гражданский брак,1,M,компаньон,0,NaN,на проведение свадьбы,NaN
17014,0,0.0,39,начальное,3,женат / замужем,0,F,госслужащий,0,NaN,операции с коммерческой недвижимостью,NaN
17646,0,0.0,49,неоконченное высшее,2,женат / замужем,0,M,компаньон,0,NaN,получение образования,NaN
17696,1,0.0,45,начальное,3,женат / замужем,0,F,пенсионер,0,NaN,операции с недвижимостью,NaN


In [13]:
# в столбце 'total_income' также остались значения с "ошибкой"
statistic_score[statistic_score['total_income'] == 'ошибка'].count()

children               4
days_employed          4
dob_years              4
education              4
education_id           4
family_status          4
family_status_id       4
gender                 4
income_type            4
debt                   4
total_income           4
purpose                4
median_total_income    4
dtype: int64

In [14]:
statistic_score = statistic_score.dropna()
statistic_score = statistic_score[statistic_score['total_income'] != 'ошибка']

Значения столбца days_employd (общий трудовой стаж в днях) не скажутся на проверямых нами гипотезах.
Вероятно, данные значения были пропущенны или не указаны при оформлении клиентской базы данных, пропуски - случайны.
Заменим их на значение 0.



In [15]:
statistic_score['total_income'] != 'ошибка'

0        True
1        True
2        True
3        True
4        True
         ... 
21520    True
21521    True
21522    True
21523    True
21524    True
Name: total_income, Length: 21511, dtype: bool

In [16]:
# проверим уникальные значения после обработки таблицы
for column in columns_unique:
    print(column, statistic_score[column].unique())
    print()

children [ 1  0  3  2  4 20  5]

days_employed [8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]

education ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']

education_id [0 1 2 3 4]

family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']

family_status_id [0 1 2 3 4]

gender ['F' 'M' 'unknown']

income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный' 'студент'
 'предприниматель' 'в декрете']

debt [0 1]

total_income [253875.6394525987 112080.01410244203 145885.95229686378 ...
 89672.56115303475 244093.0505004255 82047.41889948228]

purpose ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижим

In [17]:
#еще раз проверим общую информацию о таблице
statistic_score.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21511 entries, 0 to 21524
Data columns (total 13 columns):
children               21511 non-null int64
days_employed          21511 non-null float64
dob_years              21511 non-null int64
education              21511 non-null object
education_id           21511 non-null int64
family_status          21511 non-null object
family_status_id       21511 non-null int64
gender                 21511 non-null object
income_type            21511 non-null object
debt                   21511 non-null int64
total_income           21511 non-null object
purpose                21511 non-null object
median_total_income    21511 non-null object
dtypes: float64(1), int64(5), object(7)
memory usage: 2.3+ MB


**Вывод**

Пропущенные значения в таблице: 
- days_employd - общий трудовой стаж в днях

Данные общего трудового стажа можно заменить на медианные значения в соответствии с возрастом клиентов. Однако общий трудовой стаж может быть нам не важен в свете исследования наших гипотез. Их заменили на '0'.

- total_income - ежемесячный доход

Для обработки пропусков ежемесячного дохода была найдена медиану ежемесячного дохода по соответствию с возрастом клиента, его образования и типа занятости.

Обнаруженные отрицательные значения в количестве дней трудового стажа были исправлены на абсолютные. Видимо ошибка в сохранении данных (случайная).
Отрицательное значение в количестве детей было исправлено. Видимо ошибка в записи данных (случайная).
Ошибка в сохранении пола - некорректное значение XNA - исправленно.
Выявляется различный регистр в записи степени образования, исправлено.

### Замена типа данных

In [18]:
# заменим тип данных в столбце ежемесячного дохода в вещественный тип для удобства проведения арифметических операций
statistic_score['total_income'] = pd.to_numeric(statistic_score['total_income'])
statistic_score['total_income'] = statistic_score['total_income'].astype('int')
statistic_score['days_employed'] = statistic_score['days_employed'].astype('int')
statistic_score['median_total_income'] = pd.to_numeric(statistic_score['median_total_income'])
statistic_score['median_total_income'] = statistic_score['median_total_income'].astype('int')
statistic_score.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21511 entries, 0 to 21524
Data columns (total 13 columns):
children               21511 non-null int64
days_employed          21511 non-null int64
dob_years              21511 non-null int64
education              21511 non-null object
education_id           21511 non-null int64
family_status          21511 non-null object
family_status_id       21511 non-null int64
gender                 21511 non-null object
income_type            21511 non-null object
debt                   21511 non-null int64
total_income           21511 non-null int64
purpose                21511 non-null object
median_total_income    21511 non-null int64
dtypes: int64(8), object(5)
memory usage: 2.3+ MB


**Вывод**

Получили однородную таблицу по типам данных в столбцах, теперь удобно выполнять с ними различные целевые манипуляции

### Обработка дубликатов

In [19]:
# проверим количество явных дубликатов
print('Количество явных дубликатов до обработки:', statistic_score.duplicated().sum())

Количество явных дубликатов до обработки: 71


In [20]:
# удалим явные дубликаты, для этого воспользуемся методом drop_duplicated() со сбросом индексов .reset_index()
statistic_score = statistic_score.drop_duplicates().reset_index(drop=True)
# проверим количество дубликатов после проведения операции 
print('Количество явных дубликатов после обработки:', statistic_score.duplicated().sum())
# идем дальше

Количество явных дубликатов после обработки: 0


**Вывод**

Проведена очистка явных дубликатов (в количестве 71). Видимо, указанные дубликаты были занесены в таблицу в связи с повторным введением данных клиентов при оформлении новых заявок на кредиты

### Лемматизация

In [21]:
# импортируем библиотеку для лемматизации (вынесем импорт в самое начало проекта)
m = Mystem()
# для начала определим уникальные значения в столбцах, которые подвергнутся лемматизации
# это столбцы с целями получения кредитов (purpose)
print('Список уникальных значений столбца целей кредита: \n', statistic_score['purpose'].sort_values().unique())

Список уникальных значений столбца целей кредита: 
 ['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомобиля'
 'на проведение свадьбы' 'недвижимость' 'образование' 'операции с жильем'
 'операции с коммерческой недвижимостью' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'покупка жилой недвижимости'
 'покупка жилья' 'покупка жилья для сдачи' 'покупка жилья для семьи'
 'покупка коммерческой недвижимости' 'покупка недвижимости'
 'покупка своего жилья' 'получение высшего образования'
 'получение дополнительного образования' 'получение образования'
 'приобретение автомобиля' 'профильное образование' 'ремонт жилью'
 'свадьба' 'свой автомобиль' 'сделка с автомобилем'
 'сделка с подержанным автомобилем' 'строительство жилой недвижимости'
 'строительство недвижимости' 'строительство собственной недвижимос

In [22]:
# далее создадим функцию для обработки лемм
def lemmatize_purpose(row):
    lemmas = m.lemmatize(row)
    for lemm in lemmas:
        if 'авто' in ''.join(lemmas).strip():
            return 'автомобиль'
        if 'образов' in ''.join(lemmas).strip():
            return 'образование'
        if 'свадьб' in ''.join(lemmas).strip():
            return 'свадьба'
        if 'ремонт' in ''.join(lemmas).strip():
            return 'ремонт жилья'
        if 'жиль' in ''.join(lemmas).strip():
            return 'недвижимость'
        if 'недвиж' in ''.join(lemmas).strip():
            return 'недвижимость'
# поместим через метод .apply созданную функцию через дополнительную функцию lambda для получения леммы каждого столбца
# исходного списка 'purpose' и сразу запишем их в новый столбец 'clean_purpse'
statistic_score['clean_purpose'] = statistic_score.apply(lambda row: lemmatize_purpose(row['purpose']), axis=1)

In [23]:
# выведем список уникальных лемм
print('\nСписок лемм:\n', statistic_score['clean_purpose'].sort_values().unique())


Список лемм:
 ['автомобиль' 'недвижимость' 'образование' 'ремонт жилья' 'свадьба']


**Вывод**

Выделили список лемм для удобства категоризации целей полученяи кредита. Выявилось множество схожих целей, которые достаточно легко объединить в категории. Создали столбец с леммами, по которому и будет проходить категоризация. Выделили 5 категорий целей: недвижимость, образование, автомобиль, свадьба, ремонт жилья

### Категоризация данных

Категоризуем данные по количеству детей и уровню дохода 

In [24]:
# создадим функцию для определения стстуса многодетности семьи для применения ее к общему датафрейму
def child_fam_status(child):
    if child == 0:
        return 'бездетные'
    if child == 1:
        return 'однодетные'
    if child == 2:
        return 'малодетные'
    else:
        return 'многодетные'

In [25]:
# добавим столбец в общую таблицу
statistic_score['children_status'] = statistic_score['children'].apply(child_fam_status)
# выведем сводную таблицу по новой категории
children_status_table = statistic_score.pivot_table(index='children_status', columns='education', values='clean_purpose', aggfunc='count')
children_status_table

education,высшее,начальное,неоконченное высшее,среднее,ученая степень
children_status,,,,,
бездетные,3255.0,186.0,484.0,10152.0,4.0
малодетные,555.0,25.0,72.0,1399.0,NaN
многодетные,106.0,8.0,10.0,331.0,1.0
однодетные,1330.0,58.0,175.0,3288.0,1.0


In [26]:
# создадим функцию для категоризации по уровню дохода
def income_status(money):
    if money < 50000:
        return 'низкий доход'
    if 50000 <= money < 100000:
        return 'средний доход'
    if 100000 <= money < 200000:
        return 'доход выше среднего'
    else:
        return 'запредельный доход'

In [27]:
# добавим новую категорию в общую таблицу
statistic_score['income_status'] = statistic_score['total_income'].apply(income_status)
# выведем сводную таблицу по новой категории
income_status_table = statistic_score.pivot_table(index='income_status', columns='education', values='clean_purpose', aggfunc='count')
income_status_table

education,высшее,начальное,неоконченное высшее,среднее,ученая степень
income_status,,,,,
доход выше среднего,2628.0,139.0,396.0,8580.0,3.0
запредельный доход,2010.0,42.0,227.0,2933.0,2.0
низкий доход,31.0,15.0,3.0,323.0,NaN
средний доход,577.0,81.0,115.0,3334.0,1.0


In [30]:
def categorize_value(value):
    if value < 0.5:
        return 'lower'
    elif value < 0.75:
        return 'middle'
    else:
        return 'high'

In [31]:
df['category_value'] = df['value'].map(categorize_value)

**Вывод**

Добавлены новые качественные категории на основе количесственных показателей. С такими категориями будет удобно проверять гипотезы.

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
# выведем таблицу соотношения вероятности возвращения кредитов в зависимости от наличия детей
statistic_score.groupby('children_status').agg({'debt': 'mean'}).sort_values(by='debt', ascending=False)

,debt
children_status,
малодетные,0.094588
однодетные,0.091715
многодетные,0.085526
бездетные,0.075492


In [33]:
# сравним с количеством людей соответствующих групп
statistic_score.groupby('children_status').agg({'debt': 'count'}).sort_values(by='debt', ascending=False)

,debt
children_status,
бездетные,14081
однодетные,4852
малодетные,2051
многодетные,456


**Вывод**

Предположу выводы: малодетные и однодетные семьи имеют схожую и большую вероятность сложности с возвратом кредита в срок. Возможно это связано с тем, что многодетные семьи меньше оформляют кредиты. Бездетным же заемщикам легче выплатить кредит в срок.

In [35]:
# для большего эффекта сгенерим большее кол-во данных + искусственный столбец с debt
df = pd.DataFrame({
    'value': [random.random() for _ in range(100)],
    'is_debt': np.random.binomial(2, 0.1, size=(100,))
})

# категоризуем
df['category_value'] = df['value'].map(categorize_value)

In [36]:
# группируем по категориям и считаем среднее по is_debt - получаем долю по категориям
df.groupby('category_value').agg({'is_debt': 'mean'}).sort_values(by='is_debt', ascending=False)

,is_debt
category_value,
lower,0.326087
middle,0.275862
high,0.200000


На основе моей искуственно созданной таблице, видим, что доля больше у группы

In [45]:
statistic_score.groupby('children_status').agg({'debt': ['mean', 'count']})

debt       
                     mean  count
children_status                 
бездетные        0.075492  14081
малодетные       0.094588   2051
многодетные      0.085526    456
однодетные       0.091715   4852

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [37]:
# выведем таблицу соотношения вероятности возвращения кредитов в зависимости от семейного положения заемщиков
statistic_score.groupby('family_status').agg({'debt': 'mean'}).sort_values(by='debt', ascending=False)

,debt
family_status,
не женат / не замужем,0.097578
гражданский брак,0.093562
женат / замужем,0.075495
в разводе,0.071189
вдовец / вдова,0.065693


In [38]:
# сравним с количеством людей соответствующих групп
statistic_score.groupby('family_status').agg({'debt': 'count'}).sort_values(by='debt', ascending=False)

,debt
family_status,
женат / замужем,12332
гражданский брак,4147
не женат / не замужем,2808
в разводе,1194
вдовец / вдова,959


**Вывод**

Более высокая доля затруднений в возврате кредитов в срок у группы заемщиков, не состоящих в браке. Схожая позиция у семей, находящихся в т.н. "гражданском браке". Овдовевшие имеют меньший риск невыплаты кредитов в срок, вероятно, в силу невысокого кредитного заема (исключительно предположение).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
# выведем таблицу соотношения вероятности возвращения кредитов в зависимости от уровня доходов
statistic_score.groupby('income_status').agg({'debt': 'mean'}).sort_values(by='debt', ascending=False)

,debt
income_status,
доход выше среднего,0.086753
средний доход,0.081061
запредельный доход,0.070196
низкий доход,0.061828


In [40]:
# сравним с количеством взявших кредит с соответствующим уровнем доходов
statistic_score.groupby('income_status').agg({'debt': 'count'}).sort_values(by='debt', ascending=False)

,debt
income_status,
доход выше среднего,11746
запредельный доход,5214
средний доход,4108
низкий доход,372


**Вывод**

Выше вероятность невыплаты кредитных сумм у группы заемщиков с доходом выше среднего. Веротяно, это наиболее обширная группа, в которой имеется влияение различных факторов. Меньшие риски имеет группа с "запредельным доходом" и низким доходом - веротяно у первой есть возможность их выплатить, у второй - малая сумма заемных средств.

- Как разные цели кредита влияют на его возврат в срок?

In [41]:
# выведем таблицу соотношения вероятности возвращения кредитов в зависимости от цели
statistic_score.groupby('clean_purpose').agg({'debt': 'mean'}).sort_values(by='debt', ascending=False)

,debt
clean_purpose,
автомобиль,0.093612
образование,0.092223
свадьба,0.080138
недвижимость,0.073264
ремонт жилья,0.057756


In [42]:
# сравним с количеством взявших кредит на соответствующие цели
statistic_score.groupby('clean_purpose').agg({'debt': 'count'}).sort_values(by='debt', ascending=False)

,debt
clean_purpose,
недвижимость,10196
автомобиль,4305
образование,4012
свадьба,2321
ремонт жилья,606


**Вывод**

Наибольшие сложности с выплатой кредитов возникают у людей, взявших автокредит и людей, выбравших целью получение образования. Выплата кредитных средств на ремонт жилья не представляет существенной трудности, вероятно, в силу их небольших сумм.

## Шаг 4. Общий вывод

В каждой гипотезе предположил причины возникновения зависимостей, однако для более точных выводов, конечно, треубется выяснения соотношений совокупности факторов. Для этого требуется более глубой анализ имеющейся таблицы. Ну и требуются новые гипотезы.
Пока же можно подытожить следующее.

- Малодетные и однодетные семьи имеют схожую и большую вероятность сложности с возвратом кредита в срок. Бездетным же заемщикам легче выплатить кредит в срок.

- Более высокая доля затруднений в возврате кредитов в срок у группы заемщиков, не состоящих в браке. Схожая позиция у семей, находящихся в т.н. "гражданском браке". Овдовевшие имеют меньший риск невыплаты кредитов в срок.

- Существует зависимость невыплаты кредитных сумм у группы заемщиков с доходом выше среднего. Меньшие риски имеет группа с "запредельным доходом" и низким доходом - веротяно у первой из них есть возможность их выплатить, у второй - малая сумма заемных средств.

- Наибольшие сложности с выплатой кредитов возникают у людей, взявших автокредит и людей, выбравших целью получение образования. Выплата кредитных средств на ремонт жилья не представляет существенной трудности, вероятно, в силу их небольших сумм.